## 欠測値の扱い
### 欠測値表現のトレードオフ
テーブルやDataFrameに欠落しているデータが存在していることを示すための手法は大きく２つ<br>
1. 欠測値を大域的に示すマスクを使用する
2. 欠落しているエントリを示す特殊な値を使う
<br>

マスク式では、マスクを完全に別のブール値配列で表現したり、または値が欠落していることを表すために１ビットを割り当てる方法などがある。

特殊な値を使う方式では、　−９９９９や特殊なビットパターンを使って整数としての値が存在指定ないことを示すデータの規則を使用したり、
IEEE浮動小数点仕様で特殊な値を示すNaN(Not a Number)を用いて、値が存在しないことを示すこともある。

トレードオフ<br>
マスク式：マスク配列を別に用意するための演算と記憶域が必要<br>
特殊な値：表現可能な値の範囲を減らす、演算で余分なロジックを必要とする。NaNのような一般的な特殊値は全てのデータ型で使用可能とは限らない。

### pandasの欠測値
pandasは欠落しているデータに対して、特殊値を使用している。
* NaN値
* Python Noneオブジェクト

の既存の２つのPython null値である。

##### None:Pythonの欠測値

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ['object', 'int']:
    print("dtype = ", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype =  object
10 loops, best of 3: 93 ms per loop

dtype =  int
100 loops, best of 3: 2.8 ms per loop



In [4]:
# 配列要素にNone値をもつ配列全体に対してsum()またはminなどの集約演算を使うとたいていエラーになる。
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

#### NaN:数値データの欠測値

In [5]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

In [6]:
# NaNを使用した算術演算の結果はNaNになる
1 + np.nan

nan

In [7]:
0 * np.nan

nan

In [8]:
# エラーにはならないが、集約結果が有用ではない。
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

In [10]:
# 欠測値を無視する特別なメソッドが提供されている
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

NaNは浮動小数値であるため、整数、文字列、その他の型にNaN値は存在しないことに注意

### null値の検出

In [11]:
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [12]:
data[data.notnull()]

0        1
2    hello
dtype: object

### 欠測値の除外

In [13]:
data.dropna()

0        1
2    hello
dtype: object

In [15]:
df = pd.DataFrame([[1, np.nan, 2],
                                    [2, 3, 5],
                                     [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [16]:
# null値が存在する全ての行を削除する
df.dropna()

,0,1,2
1,2.0,3.0,5


In [17]:
# axis=1を指定するとnull値を含む全ての列を削除する
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [18]:
# howまたはthreshパラメータで許容するヌル数を制御できる
# デフォルトはhow='any', how='all'を指定すると全ての要素がnull値の行/列のみが削除される
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [19]:
df.dropna(axis=1, how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [20]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### 欠測値への値設定

In [21]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [22]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [23]:
# NAの要素に対して１つ前の値を埋める
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [24]:
# NAの要素に対して１つ後ろの値を埋める
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [25]:
# DataFrameの場合も同様、さらに値を埋める方向の軸を指定できる
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [26]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [27]:
df.fillna(method='ffill', axis=0)

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN
